# Data preparation and adding unique site identifier

This notebook downloads the raw Staley et al. (2016) Excel file and

- loads it into a Pandas dataframe
- converts all coordinates from UTM (different projections) to WGS84
- stores the site location as Shapely point and represents the table as Geopandas dataframe
- adds a column with a unique site identifier for each debris flow record
- saves the result as parquet file.

Download Staley et al. (2016) dataset:

In [1]:
import requests
from os.path import basename
import pandas as pd

In [2]:
url="https://pubs.usgs.gov/of/2016/1106/ofr20161106_appx-1.xlsx"
xlsfile=basename(url)

Download and save the data:

In [3]:
with open(xlsfile,"wb") as fid:
    fid.write(requests.get(url).content)

Read data into memory:

In [4]:
xl=pd.ExcelFile("ofr20161106_appx-1.xlsx")
desc=xl.parse(xl.sheet_names[0])
modelData=xl.parse(xl.sheet_names[1])

The first table sheet contains column descriptions:

In [5]:
desc.head()

,U.S. Geological Survey Open-File Report 2016-1106,Unnamed: 1
0,Appendix 1. Data supporting logistic regressio...,NaN
1,"[ID, identifier (in heading); UTM, Universal T...",NaN
2,NaN,NaN
3,Column Header,Description
4,Fire Name,Name of wildfire


In [7]:
%pip install shapely

     |████████████████████████████████| 1.4 MB 2.9 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [10]:
import shapely
shapely.__version__

'2.0.1'

In [11]:
%pip install geopandas

     |████████████████████████████████| 1.1 MB 3.5 MB/s eta 0:00:01
     |████████████████████████████████| 18.3 MB 77.8 MB/s eta 0:00:01
     |████████████████████████████████| 8.4 MB 55.3 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [12]:
from shapely.geometry import point
import geopandas as gpd

In [13]:
gpd.__version__

'0.12.2'

Defines function that creates shapely point from each set of `UTM_Zone`, `UTM_X` and `UTM_Y`.  Complicated due to having different UTM zones.  The function `GeoPandas.points_from_xy` can be used alternatively to avoid the warning about the Shapely array interface.

In [14]:
from pyproj import Proj


def utm2point(val):
    
    myproj = Proj(proj="utm", zone=int(val[0]), ellps="WGS84")
    
    lon,lat=myproj(val[1], val[2], inverse=True)
    
    
    return point.Point(lon,lat)
    

modelData["geom"]=modelData.loc[:,["UTM_Zone","UTM_X","UTM_Y"]].apply(utm2point, axis=1)

Save table as GeoDataFrame with WGS84 CRS [4326](https://epsg.io/4326), dropping UTM info:

In [15]:
geomData=gpd.GeoDataFrame(modelData.drop(columns=["UTM_Zone","UTM_X","UTM_Y"]), 
                                         crs="EPSG:4326", geometry="geom")

Transform column names to lowercase, easier for use in PostGRES later:

In [16]:
cols=[a.lower() for a in geomData.columns]
cols[0]="fire_name"
coln=[a.replace("/","") for a in cols]
geomData.columns=coln

Save latitude and longitude in separate columns (although already included in geometry column):

In [17]:
geomData["lon"] = geomData["geom"].x
geomData["lat"] = geomData["geom"].y

In [19]:
sitelocs=geomData[["lon","lat"]].drop_duplicates().copy()
sitelocs.reset_index(inplace=True, drop=True)
sitelocs.reset_index(inplace=True)
ncols=list(sitelocs.columns)
ncols[0]="SiteID"
sitelocs.columns=ncols

#carrying out an inner join based on the UTM coordinates
modelDataI=geomData.merge(sitelocs, left_on=["lon", "lat"], right_on=["lon","lat"])
modelDataI.tail()

,fire_name,year,fire_id,fire_segid,database,state,response,stormdate,gaugedist_m,stormstart,...,prophm23,dnbr1000,kf,acc015_mm,acc030_mm,acc060_mm,geom,lon,lat,SiteID
1545,Wallow,2011,wlw,wlw_47409,Test,AZ,0,2011-09-07 00:00:00,2706.25,2011-09-07 15:00:00,...,0.009801,0.187053,0.0,3.50,4.0,NaN,POINT (-109.26694 33.65498),-109.266936,33.654978,714
1546,Wallow,2011,wlw,wlw_47535,Test,AZ,0,2011-07-11 00:00:00,2891.75,2011-07-11 14:45:00,...,0.001571,0.500223,0.0,15.75,27.0,39.0,POINT (-109.27256 33.65397),-109.272564,33.653975,715
1547,Wallow,2011,wlw,wlw_47535,Test,AZ,0,2011-07-26 00:00:00,2891.75,2011-07-26 10:45:00,...,0.001571,0.500223,0.0,7.25,8.0,NaN,POINT (-109.27256 33.65397),-109.272564,33.653975,715
1548,Wallow,2011,wlw,wlw_47535,Test,AZ,0,2011-08-15 00:00:00,2891.75,2011-08-15 11:00:00,...,0.001571,0.500223,0.0,6.25,8.0,NaN,POINT (-109.27256 33.65397),-109.272564,33.653975,715
1549,Wallow,2011,wlw,wlw_47535,Test,AZ,0,2011-09-07 00:00:00,2891.75,2011-09-07 15:00:00,...,0.001571,0.500223,0.0,3.50,4.0,NaN,POINT (-109.27256 33.65397),-109.272564,33.653975,715


The parquet format complains about some of the stormdate entries.  Finding those that are not of type `datetime`:

In [20]:
sel=modelDataI["stormdate"].apply(lambda x: type(x) == str)
modelDataI.loc[sel,"stormdate"]

622    9/10-9/12/2002
626    9/10-9/12/2002
627    9/10-9/12/2002
632    9/10-9/12/2002
635    9/10-9/12/2002
Name: stormdate, dtype: object

In [21]:
import datetime

In [22]:
adate=datetime.date(2006,5,26)

In [23]:
atime=datetime.datetime(2006,5,26, 23, 00)

A function that converts these date ranges into day (based on first day of storm):

In [24]:
def fix_stormdate(instr):
    year=int(instr.split("/")[-1])
    month=int(instr.split("/")[1].split("-")[0])
    day=int(instr.split("/")[0])
    return datetime.date(year, month, day)

Applying the function to the bad entries:

In [25]:
modelDataI.loc[sel,"stormdate"] = modelDataI.loc[sel,"stormdate"].apply(fix_stormdate)

Convert all date entries from datetime to day:

In [26]:
modelDataI["stormdate"] = modelDataI["stormdate"].apply(lambda x: x.day)

Saving as parquet file:

In [31]:
modelDataI.to_parquet("staley16_debrisflow.parquet")

In [33]:
import pyarrow

In [34]:
pyarrow.__version__

'11.0.0'